In [1]:
import requests
import json

In [33]:
def request(url, params):
    data = None
    try:
        # Make the GET request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    return data


In [3]:
# https://<domain>/api/explore/v2.1/<resource>
BASE_URL = "https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records"

# Call to specicific museum
params = {
    # "limit": 5,  
    # "offset": 0, 
    # "lan": "fr"
    # "include_links": "true"
}

response = request(BASE_URL, params)

print(json.dumps(response, indent=4))

{
    "total_count": 707664,
    "results": [
        {
            "reference": "000PE020167",
            "ancien_depot": null,
            "appellation": null,
            "ancienne_appartenance": "Blanche Jacques Emile, coll. de l'artiste",
            "ancienne_attribution": null,
            "auteur": "BLANCHE Jacques Emile",
            "bibliographie": "Manciet C., Catalogue, Mus\u00e9e de Peinture et de Sculpture, 1933, (p. 55, n\u00b0 181.)#- Vergnet-Ruiz J., Laclotte M., Petits et grands Mus\u00e9es de France, Paris, 1962, (p. 227.)#- Dussol D., Le Salon des Amis des Arts de Bordeaux (1851-1939), th\u00e8se de doctorat, Universit\u00e9 Michel Montaigne-Bordeaux III, 1994, (p. 1102.)",
            "commentaires": null,
            "presence_image": "non",
            "date_d_acquisition": "1932",
            "date_de_depot": null,
            "decouverte_collecte": null,
            "denomination": "tableau",
            "lieu_de_depot": null,
            "description": null,

In [5]:
# fields in french
fields = response["results"][0]


In [13]:
# display fields in fr to copy-paste to translator 
[print(" ".join(field.split("_"))) for field in fields]

reference
ancien depot
appellation
ancienne appartenance
ancienne attribution
auteur
bibliographie
commentaires
presence image
date d acquisition
date de depot
decouverte collecte
denomination
lieu de depot
description
mesures
date de mise a jour
date creation
domaine
region
departement
date sujet represente
ecole pays
epoque
exposition
genese
geographie historique
inscription
numero inventaire
appellation musee de france
lien base arcade
lieu de creation utilisation
localisation
ville
lien video
manquant
manquant com
millesime de creation
millesime d utilisation
code museofile
nom officiel musee
genre
onomastique
precisions sur l auteur
precisions decouverte collecte
periode de l original copie
periode de creation
periode d utilisation
precisions inscriptions
precisions lieux creations
precisions sujets representes
precisions utilisation
references memoires
references merimee
reference maj
references palissy
sujet represente
lien inha
source de la representation
statut juridique
mater

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
translated_fields = """Reference
Former repository
Name
Former ownership
Former attribution
Author
Bibliography
Comments
Image presence
Acquisition date
Deposit date
Discovery collection
Name
Deposit location
Description
Measurements
Update date
Creation date
Domain
Region
Department
Date of subject represented
School country
Era
Exhibition
Genesis
Historical geography
Inscription
Inventory number
Museum of France name
Archway database link
Place of creation use
Location
City
Video link
Missing
Missing com
Creation year
Use year
Museofile code
Official museum name
Genre
Onomastic
Author details
Discovery collection details
Original copy period
Creation period
Use period
Inscription details
Creation location details
Subject represented details
Use details
References memoirs
Merimee references
Reference Update
Palissy references
Subject represented
Link to inha
Source of representation
Legal status
Technical materials
Title
Use
Link to associated site
Contact information
Artist under copyright
Date of entry into the public domain""".split("\n")

translated_fields = ["_".join(field.lower().split()) for field in translated_fields]
# [print(f) for f in translated_fields]
fr_to_eng_fields = {fr_field: eng_field for fr_field, eng_field in zip(fields, translated_fields)}


In [ ]:
# TO DO
# which fields are useful


In [ ]:
### Extract specific museum

In [16]:
#  play around extractions
# how many records are in specific museum:
field0 = "nom_officiel_musee"
value0 = "musée du Louvre"
BASE_URL = "https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records"

# Call to specicific museum
params = {
    "where": f"search({field0},\"{value0}\")"
}

response = request(BASE_URL, params)

print(json.dumps(response, indent=4))

{
    "total_count": 142090,
    "results": [
        {
            "reference": "50380018764",
            "ancien_depot": null,
            "appellation": null,
            "ancienne_appartenance": "coll. Rothschild",
            "ancienne_attribution": null,
            "auteur": "AUGUSTE Robert Joseph",
            "bibliographie": null,
            "commentaires": null,
            "presence_image": "oui",
            "date_d_acquisition": "1975/12/29 d\u00e9cision dation ; 1976 date d'acquisition",
            "date_de_depot": null,
            "decouverte_collecte": null,
            "denomination": "service de table (ensemble, 23);seau \u00e0 verre (\u00e9l\u00e9ment);paire",
            "lieu_de_depot": null,
            "description": null,
            "mesures": "H : 0,138 ; L : 0,346 ; l : 0,220.",
            "date_de_mise_a_jour": null,
            "date_creation": "2008-11-10",
            "domaine": [
                "orf\u00e8vrerie",
                "vie domestique"
 

In [19]:
response["results"][0]["titre"]

"Service du roi d'Angleterre George III : Seau à verres, d'une paire"

In [14]:
print(response["results"], indent=4)

TypeError: 'indent' is an invalid keyword argument for print()

### Extract list of distinct museums 

In [5]:
url = "https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records"
field = "nom_officiel_musee"
params = {
    "select": field,
    "group_by": field
}

response = request(url, params)
print(json.dumps(response, indent=4))

{
    "results": [
        {
            "nom_officiel_musee": null
        },
        {
            "nom_officiel_musee": "ARCH\u00c9A, Arch\u00e9ologie en Pays de France"
        },
        {
            "nom_officiel_musee": "Biblioth\u00e8que de l\u2019Assembl\u00e9e Nationale"
        },
        {
            "nom_officiel_musee": "Biblioth\u00e8que municipale de Besan\u00e7on"
        },
        {
            "nom_officiel_musee": "Biblioth\u00e8que municipale de Grasse"
        },
        {
            "nom_officiel_musee": "CAPC-mus\u00e9e d'Art Contemporain"
        },
        {
            "nom_officiel_musee": "Carr\u00e9 d'art - mus\u00e9e d'art contemporain"
        },
        {
            "nom_officiel_musee": "Ch\u00e2teau-mus\u00e9e"
        },
        {
            "nom_officiel_musee": "Collections de la Fondation de Coubertin"
        },
        {
            "nom_officiel_musee": "Ecomus\u00e9e de la Basse-Seine"
        },
        {
            "nom_officiel_musee

In [13]:
unique_museums = [entity.get("nom_officiel_musee") for entity in response["results"]]
unique_museums = unique_museums[1:]

# [m for m in unique_museums if "Musée du Louvre" in m]

### Extract unique museums

In [15]:
import unidecode

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [31]:
def preprocess_text(text):
    text = text.lower()
    text = unidecode.unidecode(text)
    text = "".join([char for char in text if char.isalnum() or char.isspace()])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words("french")]
    return " ".join(tokens)


In [32]:
preprocessed_titles = [preprocess_text(museum_name) for museum_name in unique_museums]
preprocessed_titles

['archea archeologie pays france',
 'bibliotheque lassemblee nationale',
 'bibliotheque municipale besancon',
 'bibliotheque municipale grasse',
 'capcmusee dart contemporain',
 'carre dart musee dart contemporain',
 'chateaumusee',
 'collections fondation coubertin',
 'ecomusee basseseine',
 'ethnotheque musee boucles seine normande',
 'familistere guise',
 'historial vendee',
 'laac lieu dart daction contemporaine',
 'piscine musee dart dindustrie andre diligent roubaix',
 'lam lille metropole musee dart moderne dart contemporain dart brut',
 'beffroi musee boucher perthes manessier',
 'carroi musee darts dhistoire',
 'musee arts figures pyrenees centrales',
 'abattoirs musee dart moderne contemporain',
 'pecheries musee fecamp',
 'lugdunummusee theatres romains',
 'mudo musee loise',
 'mus musee dhistoire urbaine sociale suresnes',
 'muba eugene leroy tourcoing',
 'maison victor hugo',
 'maison larcheologie vosges nord',
 'maison lumieres denis diderot',
 'maison verre cristal',
 'm

### Compute similarity

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [34]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_titles)

similarity_matrix = cosine_similarity(tfidf_matrix)
print(similarity_matrix)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.3061462  ... 0.         0.         0.        ]
 [0.         0.3061462  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.45142483 0.53075849]
 [0.         0.         0.         ... 0.45142483 1.         0.52531495]
 [0.         0.         0.         ... 0.53075849 0.52531495 1.        ]]


In [45]:
similarity_matrix_normalized = (similarity_matrix + 1)/2


In [52]:
distance = 1 - similarity_matrix_normalized
distance = np.maximum(0, distance)

In [36]:
from sklearn.cluster import DBSCAN
import numpy as np

In [56]:
similarity_threshold = 0.7
distance_matrix = 1 - similarity_matrix_normalized
distance_matrix = np.maximum(0, distance_matrix)
clustering = DBSCAN(eps=1-similarity_threshold, min_samples=1, metric="precomputed").fit(distance_matrix)

clusters = clustering.labels_
for cluster_id, title in sorted(zip(clusters, unique_museums)):
    print(f"Cluster {cluster_id}: {title}")

Cluster 0: ARCHÉA, Archéologie en Pays de France
Cluster 1: Bibliothèque de l’Assemblée Nationale
Cluster 2: Bibliothèque municipale de Besançon
Cluster 2: Bibliothèque municipale de Grasse
Cluster 3: CAPC-musée d'Art Contemporain
Cluster 3: Carré d'art - musée d'art contemporain
Cluster 3: LaM, Lille Métropole musée d'art moderne, d'art contemporain et d'art brut
Cluster 3: Les Abattoirs, musée d'art moderne et contemporain
Cluster 3: MUDO - musée de l'Oise
Cluster 3: Musée d'Art Moderne André Malraux - MuMa
Cluster 3: collections du musée de la société archéologique de Touraine
Cluster 3: hôtel Cabu, musée d’histoire et d’archéologie
Cluster 3: les Abattoirs, musée d'art moderne et contemporain
Cluster 3: musée Antoine Lécuyer
Cluster 3: musée Antoine Vivenel
Cluster 3: musée Archéologique départemental du Val-d'Oise
Cluster 3: musée Bonnat-Helleu, musée des Beaux-Arts de Bayonne
Cluster 3: musée Municipal
Cluster 3: musée Municipal d'Art et d'Histoire
Cluster 3: musée Saint-Raymond


In [ ]:
"https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records?where=search(nom_officiel_musee,%20%22mus%C3%A9e%20de%20l%27Ecole%20nationale%20sup%C3%A9rieure%20des%20beaux-arts%22)"
"https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records?where=search(nom_officiel_musee,mus%C3%A9e%20de%20l%27Ecole%20nationale%20sup%C3%A9rieure%20des%20beaux-arts"
"https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records?where=search(nom_officiel_musee,___%22mus%C3%A9e%20de%20l%27Ecole%20nationale%20sup%C3%A9rieure%20des%20beaux-arts%22"


In [37]:
BASE_URL + "?where=" + params["where"]

'https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records?where=search(nom_officiel_musee,mus%C3%A9e%20de%20l%27Ecole%20nationale%20sup%C3%A9rieure%20des%20beaux-arts)'

In [ ]:
specific_museum = "musée de l'Ecole nationale supérieure des beaux-arts"
BASE_URL = "https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records"

# Call to specicific museum
params = {
    # "limit": 5,  
    # "offset": 0, 
    # "nom_officiel_musee": specific_museum,
    "search": f"(nom_officiel_musee, \"{'+'.join(specific_museum.split())}\""
    # "lan": "fr"
}

response = request(BASE_URL, params)

In [24]:
print(f"(nom_officiel_musee, \"{'+'.join(specific_museum.split())}\"")

(nom_officiel_musee, "musée+de+l'Ecole+nationale+supérieure+des+beaux-arts"


# Web-scraping

## 1. musée du Louvre 

In [22]:
# Find a piece "Service du roi d'Angleterre George III : Seau à verres, d'une paire"


In [63]:
BASE_URL = "https://collections.louvre.fr"
SEARCH = "recherche"
QUERY = "q"


In [46]:
from bs4 import BeautifulSoup

In [66]:
def search(query):
    """Search for Louvre's collection"""
    urls_to_results = []
    try:
        response = requests.get(url=f"{BASE_URL}/{SEARCH}", 
                                params={QUERY: query})
        soup = BeautifulSoup(response._content, "html.parser")
        results_container = soup.find("div", class_ = "search__results__container")
        results = results_container.find_all("div", class_="card__outer")
        urls_to_results = [f"{BASE_URL}/{r.find("a").attrs.get("href")}" for r in results]
    except:
        ...
    finally:
        return urls_to_results


In [ ]:
def search(query):
    """Search for Louvre's collection"""
    urls_to_results = []
    try:
        response = requests.get(url=f"{BASE_URL}/{SEARCH}", 
                                params={QUERY: query})
        soup = BeautifulSoup(response._content, "html.parser")
        results_container = soup.find("div", class_ = "search__results__container")
        results = results_container.find_all("div", class_="card__outer")
        urls_to_results = [f"{BASE_URL}/{r.find("a").attrs.get("href")}" for r in results]
    except:
        ...
    finally:
        return urls_to_results


In [69]:
piece = "Service du roi d'Angleterre George III : Seau à verres, d'une paire"
results = search(piece)

In [174]:
# example of one of the Louvre's pieces
"""
https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records?where=search%28titre%2C%22Service+du+roi+d%27Angleterre+George+III+%3A+Seau+%C3%A0+verres%2C+d%27une+paire%22%29
"""

'\nhttps://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/base-joconde-extrait/records?where=search%28titre%2C%22Service+du+roi+d%27Angleterre+George+III+%3A+Seau+%C3%A0+verres%2C+d%27une+paire%22%29\n'

In [207]:
def get_piece_info(url):
    """returns dictionary of:
    - title
    - image resource url
    - author
    - materials
    - technique
    - date of creation
    - place of creation
    - physical location 
    - department of Louvre museum
    - description
    - historical information"""
    piece_info = None
    try:
        resource = requests.get(url)
        soup = BeautifulSoup(resource.content, "html.parser")
        content = soup.find("div", attrs={"id": "content"})

        piece_title = content.find(class_="notice__title h_1").text.strip()
        # image resource url
        piece_image_url = f"{BASE_URL}/{content.find("picture").find("img").attrs.get("src")}"
        # author
        piece_author = content.find(class_ = "notice__author").text.strip()
        # physical location
        piece_georeference = content.find(class_ = "notice__date geo-reference").text.strip() 
        # department of Louvre museum
        museum_department = content.find_all(class_ = "notice__date")[-1].text.strip() # in joconde
        
        piece_details = content.find_all("div", class_="notice__fullcartel__part")

        # # - materials, technique
        # technical_fields = [field.text.strip() for field in piece_details[2].find_all("div", class_ = "part__label")]
        # technical_values = [field.text.strip() for field in piece_details[2].find_all("div", class_ = "part__content")]
        # technical_info = {field: value for field, value in zip(technical_fields, technical_values)}
        # piece_material, piece_technique = technical_info.get('Matière et technique').split("\n")
        # # - period of creation, place of creation
        # creation_fields = [field.text.strip() for field in piece_details[3].find_all("div", class_ = "part__label")]
        # creation_values = [field.text.strip() for field in piece_details[3].find_all("div", class_ = "part__content")]
        # creation_info = {field: value for field, value in zip(creation_fields, creation_values)}
        # piece_creation_date = creation_info.get("Date de création / fabrication")
        # piece_creation_location = creation_info.get("Lieu de création / fabrication / exécution")
        # # - description
        # piece_details = content.find_all("div", class_="notice__fullcartel__part")
        # description_fields = [field.text.strip() for field in piece_details[1].find_all("div", class_ = "part__label")]
        # description_values = [field.text.strip() for field in piece_details[1].find_all("div", class_ = "part__content")]
        # description = {field: value for field, value in zip(description_fields, description_values)}
        # piece_description = description.get("Description / Décor")
        # # - historical information
        # historical_fields = [field.text.strip() for field in piece_details[4].find_all("div", class_ = "part__label")]
        # historical_values = [field.text.strip() for field in piece_details[4].find_all("div", class_ = "part__content")]
        # historical_info = {field: value for field, value in zip(historical_fields, historical_values)}
        # piece_historical_info = historical_info.get("Historique de l'œuvre")        

        detailed_info = dict()
        for i, _ in enumerate(['description', 'technical_info', 'creation', 'history'], start=1):
            fields = [field.text.strip() for field in piece_details[i].find_all("div", class_ = "part__label")]
            values = [field.text.strip() for field in piece_details[i].find_all("div", class_ = "part__content")]

            for f, v in zip(fields, values):
                detailed_info[f] = v
        
        piece_material, piece_technique = detailed_info.get("Matière et technique").split("\n")
        piece_info = {
            "title": piece_title,
            "image_url": piece_image_url,
            "author": piece_author,
            "materials": piece_material, 
            "technique": piece_technique, 
            "creation_date": detailed_info.get("Date de création / fabrication"), 
            "creation_location": detailed_info.get("Lieu de création / fabrication / exécution"), 
            "physical_location": piece_georeference,
            "department": museum_department,
            "description": detailed_info.get("Description / Décor"), 
            "history": detailed_info.get("Historique de l'œuvre")
        }
    except:
        ...
    finally:
        return piece_info

In [ ]:
url = 'https://collections.louvre.fr//ark:/53355/cl010111604'
resource = requests.get(url)
soup = BeautifulSoup(resource.content, "html.parser")
content = soup.find("div", attrs={"id": "content"})
piece_title = content.find(class_="notice__title h_1").text.strip()
# image resource url
piece_image_url = f"{BASE_URL}/{content.find("picture").find("img").attrs.get("src")}"
# author
piece_author = content.find(class_ = "notice__author").text.strip()
# physical location
piece_georeference = content.find(class_ = "notice__date geo-reference").text.strip() 
# department of Louvre museum
museum_department = content.find_all(class_ = "notice__date")[-1].text.strip() # in joconde

piece_details = content.find_all("div", class_="notice__fullcartel__part")
 
detailed_info = dict()
for i, _ in enumerate(['description', 'technical_info', 'creation', 'history'], start=1):
    fields = [field.text.strip() for field in piece_details[i].find_all("div", class_ = "part__label")]
    values = [field.text.strip() for field in piece_details[i].find_all("div", class_ = "part__content")]
    for f, v in zip(fields, values):
        detailed_info[f] = v
    
piece_material, piece_technique = detailed_info.get("Matière et technique").split("\n")

In [211]:
get_piece_info('https://collections.louvre.fr//ark:/53355/cl010111604')